In [1]:
import geopandas as gpd
import pandas as pd
import wget
import os
import glob

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
gdf_distritos = gpd.read_file('dados/SIRGAS_GPKG_distrito.gpkg')

In [3]:
gdf_distritos

,ds_areamt,ds_subpref,ds_sigla,ds_nome,ds_areakm,ds_cd_sub,ds_codigo,geometry
0,None,SANTANA-TUCURUVI,MAN,MANDAQUI,None,05,51,"POLYGON ((330950.373 7407837.176, 330952.177 7..."
1,None,MOOCA,MOO,MOOCA,None,25,53,"POLYGON ((338349.356 7394264.944, 338362.239 7..."
2,None,CASA VERDE-CACHOEIRINHA,LIM,LIMAO,None,04,50,"POLYGON ((329079.913 7402343.759, 329081.724 7..."
3,None,JABAQUARA,JAB,JABAQUARA,None,15,38,"POLYGON ((331924.648 7385714.896, 331932.746 7..."
4,None,CIDADE ADEMAR,CAD,CIDADE ADEMAR,None,16,22,"POLYGON ((332410.602 7378610.746, 332404.612 7..."
...,...,...,...,...,...,...,...,...
91,None,GUAIANASES,LAJ,LAJEADO,None,28,96,"POLYGON ((357579.815 7394795.986, 357572.551 7..."
92,None,LAPA,LAP,LAPA,None,08,48,"POLYGON ((323721.814 7398433.381, 323730.119 7..."
93,None,SE,LIB,LIBERDADE,None,09,49,"POLYGON ((332502.702 7392061.729, 332502.811 7..."
94,None,LAPA,JRE,JAGUARE,None,08,41,"POLYGON ((322952.516 7394978.618, 323250.620 7..."


In [10]:
for distrito in gdf_distritos.iterrows():

    url = f'https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_{distrito[1].ds_codigo.rjust(2, "0")}_{distrito[1].ds_nome.replace(" ", "_")}.gpkg'
    file = f'2022/SIRGAS_IPTU_2022_{distrito[1].ds_codigo.rjust(2, "0")}_{distrito[1].ds_nome.replace(" ", "_")}.gpkg'
    print(url)
    
    if not os.path.exists(file):
        _ = wget.download(url, file)

https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_51_MANDAQUI.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_53_MOOCA.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_50_LIMAO.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_38_JABAQUARA.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_22_CIDADE_ADEMAR.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_37_ITAQUERA.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_47_JOSE_BONIFACIO.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_72_SAO_LUCAS.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_56_PARI.gpkg
https://github.com/cem-usp/dash-iptu/raw/main/iptu_por_pontos/2022/SIRGAS_IPTU_2022_57_P

In [11]:
tipos_de_uso = [
    "Loja",
    "Loja e residência (predominância comercial)",
    "Outras edificações de uso comercial, com utilização múltipla",
    "Loja em edifício em condomínio (unidade autônoma)",
    "Armazéns gerais e depósitos",
    "Outras edificações de uso especial, com utilização múltipla"
]

In [16]:

if not os.path.exists('resultados/estabelecimentos_comerciais.gpkg'):
    
    distritos = []  
    
    for file in glob.glob('2022/*'):
        gdf_distrito = gpd.read_file(file)
        filtro_usos_comerciais = gdf_distrito.loc[:, 'TIPO DE USO DO IMOVEL'].isin(tipos_de_uso)
        gdf_distrito = gdf_distrito.loc[filtro_usos_comerciais, :].reset_index()
        print(file, end=', ')
        distritos.append(gdf_distrito)

    gdf_estabelecimentos_comerciais = pd.concat(distritos)

    gdf_estabelecimentos_comerciais.to_file('resultados/estabelecimentos_comerciais.gpkg', driver='GPKG')

In [17]:
gdf_hexagonos = gpd.read_file('dados/sp_grid500_dados.zip')

In [21]:
gdf_hexagonos.to_crs(epsg=31983, inplace=True)

In [61]:
# gdf_hexagonos = gpd.read_file('dados/Sao-Paulo-hexagonos-250.gpkg')

In [22]:
gdf_esta_hexagon = gdf_estabelecimentos_comerciais.sjoin(gdf_hexagonos)

In [23]:
# gdf_esta_hexagon.columns

In [24]:
atributos = [
    'index_right',
    'AREA CONSTRUIDA'
]

In [25]:
def median(x):
    return x.quantile(0.5)

def q25(x):
    return x.quantile(0.25)

def q75(x):
    return x.quantile(0.75)

In [28]:
gdf_esta_hexagon_grouped = gdf_esta_hexagon.loc[:, atributos].groupby(by='index_right').agg(
    {'AREA CONSTRUIDA': ['count', 'min', 'max', 'mean', 'std', 'var', median, q25, q75]}
)
gdf_esta_hexagon_grouped

AREA CONSTRUIDA                                              \
                      count     min     max           mean          std   
index_right                                                               
18                        3     450    8005    3118.333333  4237.954499   
28                        2    1174    1501    1337.500000   231.223917   
40                        2    1610    1815    1712.500000   144.956890   
41                        1     585     585     585.000000          NaN   
66                        1  252694  252694  252694.000000          NaN   
...                     ...     ...     ...            ...          ...   
7423                     14      26     900     234.071429   273.647574   
7425                      8      30     655     336.250000   190.415148   
7426                     17      40     911     302.235294   246.289091   
7427                     36      57    1130     311.444444   233.080176   
7428                      1      90      90      90.000000          NaN   

                                                           
                      var    median        q25        q75  
index_right                                                
18           1.796026e+07     900.0     675.00    4452.50  
28           5.346450e+04    1337.5    1255.75    1419.25  
40           2.101250e+04    1712.5    1661.25    1763.75  
41                    NaN     585.0     585.00     585.00  
66                    NaN  252694.0  252694.00  252694.00  
...                   ...       ...        ...        ...  
7423         7.488299e+04     133.5      84.75     197.50  
7425         3.625793e+04     325.0     236.00     424.25  
7426         6.065832e+04     279.0     110.00     361.00  
7427         5.432637e+04     250.0     150.00     376.50  
7428                  NaN      90.0      90.00      90.00  

[3923 rows x 9 columns]

In [32]:
gdf_estab_estatistica = gdf_hexagonos.join(gdf_esta_hexagon_grouped)

/tmp/ipykernel_181205/3207569364.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left, 2 on the right)
  gdf_estab_estatistica = gdf_hexagonos.join(gdf_esta_hexagon_grouped)


In [35]:
def name_column(x):
    if isinstance(x, tuple):
        return f'area_construida_{x[1]}'
    else:
        return x

In [37]:
gdf_estab_estatistica.columns = gdf_estab_estatistica.columns.map(lambda x: name_column(x))

In [45]:
gdf_estab_estatistica.columns

Index(['id', 'ID_2', 'CD_GEOC', 'NM_MUNI', 'pop', 'income', 'chl_EF1',
       'chl_EF2', 'yong_EM', 'hi_wh', 'hi_bl', 'hi_ot', 'md_wh', 'md_bl',
       'md_ot', 'lo_wh', 'lo_bl', 'lo_ot', 'geometry', 'area_construida_count',
       'area_construida_min', 'area_construida_max', 'area_construida_mean',
       'area_construida_std', 'area_construida_var', 'area_construida_median',
       'area_construida_q25', 'area_construida_q75'],
      dtype='object')

In [48]:
gdf_estab_estatistica.drop(
    ['id', 'ID_2'],
    axis=1
).fillna(0.0).to_file('resultados/estabelecimentos_estat_descritiva_area_construida.gpkg', driver='GPKG')

/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
